In [1]:
import netket as nk
import matplotlib.pyplot as plt
import numpy as np
from netket import experimental as nkx
from deepnets.net.ConvNext import ConvNext
from deepnets.net.ConvNext.heads import OutputHead
from deepnets.system import Shastry_Sutherland
from netket.nn.blocks import SymmExpSum
from deepnets.nn.blocks import FlipExpSum
import cmath
import seaborn as sns
import h5py
import optax
import jax
import flax
import optax
import sys
sys.path.append('..')

/opt/homebrew/Caskroom/miniforge/base/envs/netket_pro_local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Parameters
#Model parameters
L = 6
J = [0.8,1.0]
#sign_rule = [False]
#Network parameters
#ConvNext
n_blocks = (1,)
features = (12,)
expansion_factor = 2
downsample_factor = 2 #linear_patch_size 
kernel_size = (L//downsample_factor,L//downsample_factor)
output_head = OutputHead
final_features = features[0]
#Optimization parameters
samples = 1000
discard_fraction=0.0
iters = 50
diag_shift=1e-4
lr = 1e-2
r = 1e-6
SR_solver = nk.optimizer.solver.pinv_smooth(rcond=r,rcond_smooth=r)
#Scheduler parameters
T = iters
alpha = lr #minimum value
p = 1 #exponent

In [3]:
#Setup
system = Shastry_Sutherland(L = L, J = J)
lr_scheduler = optax.constant_schedule(lr)
#lr_scheduler = optax.cosine_decay_schedule(init_value = lr, decay_steps = T, alpha = alpha, exponent = p)
network = ConvNext(lattice_shape = tuple(2*system.graph.extent), n_blocks = n_blocks, features = features, expansion_factor = expansion_factor, Head = output_head, kernel_size = kernel_size, downsample_factor = downsample_factor, final_features = final_features, extract_patches = system.extract_patches_as2D)
sampler = nk.sampler.MetropolisExchange(system.hilbert_space,graph=system.graph,sweep_size=system.graph.n_nodes,d_max=1)
var_state = nk.vqs.MCState(sampler,model=network,n_samples=samples,n_discard_per_chain=discard_fraction*samples)
print(f"Number of parameters: {var_state.n_parameters}")

Number of parameters: 1248


/opt/homebrew/Caskroom/miniforge/base/envs/netket_pro_local/lib/python3.12/site-packages/netket/vqs/mc/mc_state/state.py:248: UserWarning: n_samples=1000 (1000 per device/MPI rank) does not divide n_chains=16, increased to 1008 (1008 per device/MPI rank)
  self.n_samples = n_samples


In [4]:
#Run
#scheduler = optax.schedules.cosine_decay_schedule(init_value = lr,decay_steps=iters/2)
optimizer = nk.optimizer.Sgd(learning_rate=lr_scheduler)
log=nk.logging.RuntimeLog()
if var_state.n_parameters > samples: #use minSR
    print("n_params > samples : Using minSR")
    gs = nkx.driver.VMC_SRt(system.hamiltonian, optimizer,linear_solver_fn=SR_solver,diag_shift=diag_shift, variational_state=var_state,jacobian_mode="complex")
else:
    print("n_params < samples : Using SR")
    preconditioner = nk.optimizer.SR(qgt=nk.optimizer.qgt.QGTJacobianDense,solver=SR_solver,diag_shift=diag_shift,mode="complex")
    gs = nk.VMC(system.hamiltonian, optimizer, variational_state=var_state,preconditioner=preconditioner)

gs.run(n_iter=iters,out=log,show_progress=True,timeit=False)

n_params > samples : Using minSR


 16%|█▌        | 8/50 [02:31<13:15, 18.94s/it, Energy=-33.40+0.23j ± 0.39 [σ²=156.16, R̂=1.0105]]


KeyboardInterrupt: 